# APIs

Because I didn't want to sign up for the free API account the course recommends, I've copied and pasted the material without running the code so I can still retain the information.

APIs usually work by first getting an ID and other credentials set up. Then, you can request data through an endpoint.

Here's a simple example:

In [ ]:
import requests

APP_ID = "72dba35060b54cf9ad3ffbdc68de9174"
ENDPOINT = "https://openexchangerates.org/api/latest.json"

response = requests.get(f"{ENDPOINT}?app_id={APP_ID}")
exchange_rates = response.json()

usd_amount = 1000
gbp_amount = usd_amount * exchange_rates['rates']['GBP']

print(f"USD{usd_amount} is GBP{gbp_amount}")

Creating a client is common because it helps us parse the data we're receiving. Create another file that will take care of that for our application.

When you're coding interactions with things that aren't part of your application, they are commonly stored in a file called `libs`:

In [ ]:
# libs/openexchange.py

import requests
import functools
from cachetools import cached, TTLCache


class OpenExchangeClient:
    BASE_URL = "https://openexchangerates.org/api/"

    def __init__(self, app_id):
        self.app_id = app_id
    
    @property
    @cached(cache=TTLCache(maxsize=2, ttl=900))
    def latest(self):
        return requests.get(f"{self.BASE_URL}/latest.json?app_id={self.app_id}").json()
    
    def convert(self, from_amount, from_currency, to_currency):
        rates = self.latest['rates']
        to_rate = rates[to_currency]

        if from_currency == 'USD':
            return from_amount * to_rate
        else:
            from_in_usd = from_amount / rates[from_currency]
            return from_in_usd * to_rate

In [ ]:
# app.py

from libs.openexchange import OpenExchangeClient

APP_ID = "72dba35060b54cf9ad3ffbdc68de9174"

client = OpenExchangeClient(APP_ID)

usd_amount = 1000
gbp_amount = client.convert(usd_amount, 'USD', 'GBP')

print(f"USD{usd_amount} is GBP{gbp_amount}")

Note the addition of `@cached(cache=TTLCache(maxsize=2, ttl=900))`. A **cache** stores data temporarily so it can be used again quickly. This line ensures we will remember the arguments and what they return so that if they're used again, the function doesn't have to recalculate and it can return the same thing it did last time.

We are using cachetools, which persists how many second should elapse until it stops using the cache.